# Scraping b-ok site (name of books , urls) using selenium , then download all books 
This part is used to download pdfs from b-ok website.
Unfortunately, i could only download only 5 book cause we must have an account on the website.
if we have an account we can download 10 books but i couldn't sign in while scrapping

In [0]:
#Importing packages
!pip install selenium
from selenium import webdriver
import pandas as pd
import os 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
#from fake_useragent import UserAgent
import pandas as pd
from collections.abc import Iterable
import warnings
# from pymongo import MongoClient
# from collections.abc import Iterable
import time

In [0]:
selenium_driver_path = "C:/Users/SMSM-TECH/Downloads/chromedriver_win32/chromedriver.exe"

In [0]:
# get unique elements from a list
def unique(data):
    return list(dict.fromkeys(data))


# get selenium driver object
def get_selenium_driver():
    """
    This function returns the selenium driver object.

    Parameters:
        None

    Returns:
        driver: selenium driver object
    """

    driver = webdriver.Chrome(executable_path = selenium_driver_path)

    return driver


# get BeautifulSoup object
def get_soup(url):
    """
    Given the url of a page, this function returns the soup object.
    
    Parameters:
        url: the link to get soup object for
    
    Returns:
        soup: soup object
    """
    driver = get_selenium_driver()

    driver.get(url)

    driver.implicitly_wait(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    driver.close()

    return soup

#save to MongoDB
def saveToDB(collection, city_areas_zones_dict):
    
    try:
        print("collection.count_documents({}): ",  collection.count_documents({}))
        resultSet = collection.count_documents({"City":city_areas_zones_dict["City"],"Areas_Zones":city_areas_zones_dict['Areas_Zones']})

        if resultSet > 0:
            print("city_areas_zones_dict is already scrapped >> ", city_areas_zones_dict)
            print("Skip!")
        else:
            print("New city_areas_zones_dict will be scrapped >> ", city_areas_zones_dict)
            print("I will insert it!")
            collection.insert_one(city_areas_zones_dict)
            
    except Exception as e:
        print('Exception : %s' % str(e))

In [0]:
import re

# scrape b-ok website

def scrape_books():
    books_names = []

    for num in range(1,2):
        base_url = "https://b-ok.cc/s/natural%20language%20processing?page="+str(num)

        home_page_soup = get_soup(base_url)

        # get a get_selenium_driver object
        driver = get_selenium_driver()
        #driver.execute_script("return navigator.userAgent")
        driver.get(base_url)
        

        home_page_soup=home_page_soup.find_all("div",{'id':"searchResultBox"})
        links=home_page_soup[0].find_all("a",{"href":re.compile("^/book/")})
        #print(links)
            

        all_link = []
        for link in links:
            part_of_link = link.get("href")
            link = "https://b-ok.cc"+part_of_link
            all_link.append(link)
            all_link = list(set(all_link))
            print("all_link")
            print(len(all_link))

            print(all_link)
        
        for link in all_link:
            page_driver2 = get_selenium_driver()
            page_driver2.get(link)
            page_driver2.find_element_by_class_name("dlButton").click()
            


        
        time.sleep(3)
        print("----------------------------------")
    driver.close()
        


In [0]:
scrape_books()

all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
1
['https://b-ok.cc/book/3681492/07d64c']
all_link
2
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
2
['https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
3
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
3
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/3681492/07d64c']
all_link
4
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3681492/07d64c']
all_link
4
['https://b-ok.cc/book/3711505/debacd', 'https://b-ok.cc/book/3642068/da7d8c', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3681492/07d64c']
all_link
5
['https://b-ok.cc/book/755716/be52d7', 'https://b-ok.cc/book/4984922/b162b3', 'https://b-ok.cc/book/3711505/debac

ElementClickInterceptedException: Message: element click intercepted: Element <a href="#" id="dbx_3362289" class="btn btn-primary dlButton disabled" data-toggle="tooltip" data-autoopen="true" data-placement="top" title="" data-original-title="Link deleted by legal owner (But you can use <a href=&quot;/msgn/tor&quot; target=&quot;_blank&quot; rel=&quot;nofollow&quot;>TOR</a> to download it anyway)" aria-describedby="tooltip105214">...</a> is not clickable at point (142, 604). Other element would receive the click: <div class="book-details-button">...</div>
  (Session info: chrome=80.0.3987.149)


# PDF Page Summarization Steps
1. Reading the Page content then printing it.
2. Cleaning the Page content then printing it.
3. Splitting into sentences (Tokenization) but the last sentence is removed and also the sentences which smaller than 5 words then printing all the chosen sentences.
4. Vectorizing each sentence using TF-Hub Universal sentence Encoder (embedding matrix).
5. Applaying Kmeans clusturing to cluster the sentences into certain no. of clusters.
6. Forming Page summary from the sentences which are the nearst to clusteres centroids. from each cluster we take a sentence; this sentence is the closest to the centroid. Then we join these sentences together to form the summary.
5. Printing the summary of The Page.

###### note: this notebook needs to be ran on colab as it uses the GPU version of tensorflow or you need to :
  1. Install the latest Tensorflow version.
     - !pip3 install --upgrade tensorflow-gpu
  2. Install TF-Hub by
     - !pip3 install tensorflow-hub 
     - !pip3 install seaborn

In [1]:
#installing pdfminer
!pip install PyPDF2

In [2]:
# Importing the required libraries
import PyPDF2
from PyPDF2 import PdfFileReader
import os
import tensorflow_hub as hub
import nltk
import nltk
nltk.download('punkt')
import re
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## PDF file and PDF reader objects

In [0]:
# Creating a pdf file object
pdf = open("Applied Text Analysis with Python Enabling Language-Aware Data Products with Machine Learning by Benjamin Bengfort, Tony Ojeda, Rebecca Bilbro (z-lib.org).pdf","rb")
# required variables
ENCRYPTED_FILE_PATH = "Doc1 password.pdf"
FILE_OUT_PATH = "Doc2 password.pdf"
#creating pdf reader object
pdf_reader = PdfFileReader(pdf) 

if pdf_reader.isEncrypted:
    try:
        pdf_reader.decrypt('123456')
    except NotImplementedError:
        command=f"qpdf --password='{123456}' --decrypt {ENCRYPTED_FILE_PATH} {FILE_OUT_PATH};"
        os.system(command)            
        with open("Doc1 password.pdf", mode='rb') as fp:
            pdf_reader = PyPDF2.PdfFileReader(fp)
            print(f"Number of page: {pdf_reader.getNumPages()}")

## Calculating number of pages

In [4]:
#checking number of pages in a pdf file
no_of_pages = pdf_reader.numPages
print("Num of pages in the PDF: ", no_of_pages,"\n")
            

Num of pages in the PDF:  332 



## Loading TF unversal sentence embedding pretrained model & defining the embedding function

In [5]:
   
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


## One page Summarization


### Defining a function to clean a page

In [0]:
def Clean_Page(pageText):
  # replacing multiple spaces by single space
  pageText = re.sub('[\s]+', ' ', pageText)
  # replacing multiple newlines by space
  pageText = re.sub('[\n]+', ' ', pageText)
  # replacing e.g. by for example 
  pageText = pageText.replace("e.g.","for example")
  # replacing certain symbols by space
  pageText = re.sub("\n|—|-"," ", pageText)
  # removing urls
  pageText = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',pageText)
  # removing the rest of punctuations
  pageText = re.sub("[^a-zA-Z0-9\.\s+,]","", pageText)
  # replacing multiple spaces by single space
  pageText = re.sub('[\s]+', ' ', pageText)
  return pageText

### Defining a function to summerize a page

In [0]:
def Summarize_Page(pdf_reader, pageNum):
  # creating a page object
  page = pdf_reader.getPage(pageNum)
  # extracting text from the page
  pageText = page.extractText()
  # cleaning the page text
  pageText = Clean_Page(pageText)
  print("Page Content After Cleaning:\n")
  print(pageText,"\n\n")
  # splitting the text into a list of sentences (tokenization)
  sent = pageText.split(".")####
  sentences=[]
  for i in range(len(sent)):
    if ((len(sent[i].strip().split(" "))> 4) & (i != (len(sent)-1))):###
     sentences.append(sent[i].strip())
  print("The splitted sentences:\n")
  for i in range(len(sentences)):
     print(sentences[i]+".\n")

  # embedding all the sentences in the list(vectorization)
  sen_vectors= embed(sentences)
  # from tensors to list of vectors (embedding matrix)
  sen_vectors= np.array(sen_vectors).tolist()
  # clustering using kmeans 
  # defining the number of clusters as the ceil of 2/3 from the original length of the sentence
  n_clusters = int(np.ceil(len(sentences)**0.5))
  kmeans = KMeans(n_clusters=n_clusters, init = 'k-means++', random_state = 42)
  kmeans = kmeans.fit(sen_vectors)
  # obtaining the indices of the nearst sentences to clusters centroids
  closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sen_vectors)
  # sorting the indices
  closest.sort()
  # joining the sentences (closest to centroids) to obtain the summary
  summary=[]
  for i in closest:
      summary.append(sentences[i])
  summary='. '.join(summary) + "."
  return summary

### Summarize page no. 75 (Index 74)

#### Asking the user to enter the page number

In [8]:
flag = True
while flag:
 pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 if pageNumber > (no_of_pages-1):
   print("\nInvalid page numer\n")
   pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 else:
   flag= False

Please enter the page number (from 0 to 331): 74


#### Printing the Page Content

In [9]:
# creating a page object
page = pdf_reader.getPage(pageNumber)
# extracting text from the page
pageText = page.extractText()
# printing the content of the page
print("Page Content:\n")
print(pageText)

Page Content:

CHAPTER 4Text Vectorization and TransformationPipelinesMachine learning algorithms operate on a numeric feature space, expecting input as a
two-dimensional array where rows are instances and columns are features. In order
to perform machine learning on text, we need to transform our documents into vec‡
tor representations such that we can apply numeric machine learning. This process is
called feature extraction
 or more simply, 
vectorization
, and is an essential first step
toward language-aware analysis.
Representing documents numerically gives us the ability to perform meaningful ana‡
lytics and also creates the 
instances
 on which machine learning algorithms operate. 
Intext analysis, instances are entire documents or utterances, which can vary in length
from quotes or tweets to entire books, but whose vectors are always of a uniform
length. Each property of the vector representation is a 
feature
. For text, features repre‡
sent attributes and properties of documen

#### Summerization, Printing Page Content After Cleaning and The splitted sentences





In [10]:
# summarization
summary = Summarize_Page(pdf_reader, pageNumber)

Page Content After Cleaning:

CHAPTER 4Text Vectorization and TransformationPipelinesMachine learning algorithms operate on a numeric feature space, expecting input as a two dimensional array where rows are instances and columns are features. In order to perform machine learning on text, we need to transform our documents into vec tor representations such that we can apply numeric machine learning. This process is called feature extraction or more simply, vectorization , and is an essential first step toward language aware analysis. Representing documents numerically gives us the ability to perform meaningful ana lytics and also creates the instances on which machine learning algorithms operate. Intext analysis, instances are entire documents or utterances, which can vary in length from quotes or tweets to entire books, but whose vectors are always of a uniform length. Each property of the vector representation is a feature . For text, features repre sent attributes and properties of d

#### Printing Page Summary

In [11]:
# printing the summary of the page
print("Page Summary:\n")
print(summary)

Page Summary:

Representing documents numerically gives us the ability to perform meaningful ana lytics and also creates the instances on which machine learning algorithms operate. Intext analysis, instances are entire documents or utterances, which can vary in length from quotes or tweets to entire books, but whose vectors are always of a uniform length. Points in space can be close together or far apart, tightly clustered or evenly dis tributed. The simplest encoding of semantic space is the bag of words model, whose primary insight is that meaning and similarity are encoded in vocabulary.


### Summarize page no. 29 (Index 28)

#### Asking the user to enter the page number

In [12]:
flag = True
while flag:
 pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 if pageNumber > (no_of_pages-1):
   print("\nInvalid page numer\n")
   pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 else:
   flag= False

Please enter the page number (from 0 to 331): 28


#### Printing the Page Content

In [13]:
# creating a page object
page = pdf_reader.getPage(pageNumber)
# extracting text from the page
pageText = page.extractText()
# printing the content of the page
print("Page Content:\n")
print(pageText)

Page Content:

type of language model is hugely influential to text analytics because it demonstrates
the basic mechanism of a language application—the use of context to guess meaning.
Language models also reveal the basic hypothesis behind applied machine learning
on text: text is predictable
. In fact, the mechanism used to score language models inan academic context, 
perplexity
, is a measure of how predictable the text is by evaluat‡
ing the entropy
 (the level of uncertainty or surprisal) of the language model†s proba‡
bility distribution.Consider the following partial phrases: –man†s best‹ƒ or –the witch flew on a‹ƒ.
These low entropy phrases mean that language models would guess –friendƒ and
–broomstick,ƒ respectively, with a high likelihood (and in fact, English speakers would
be surprised if the phrase wasn†t completed that way). On the other hand, high
entropy phrases like –I†m going out to dinner tonight with my‹ƒ lend themselves to a
lot of possibilities (–friend,ƒ –mother

#### Summerization, Printing Page Content After Cleaning and The splitted sentences

In [14]:
# summarization
summary = Summarize_Page(pdf_reader, pageNumber)

Page Content After Cleaning:

type of language model is hugely influential to text analytics because it demonstrates the basic mechanism of a language application the use of context to guess meaning. Language models also reveal the basic hypothesis behind applied machine learning on text text is predictable . In fact, the mechanism used to score language models inan academic context, perplexity , is a measure of how predictable the text is by evaluat ing the entropy the level of uncertainty or surprisal of the language models proba bility distribution.Consider the following partial phrases mans best or the witch flew on a. These low entropy phrases mean that language models would guess friend and broomstick, respectively, with a high likelihood and in fact, English speakers would be surprised if the phrase wasnt completed that way. On the other hand, high entropy phrases like Im going out to dinner tonight with my lend themselves to a lot of possibilities friend, mother, and work colle

#### Printing Page Summary

In [15]:
# printing the summary of the page
print("Page Summary:\n")
print(summary)

Page Summary:

Language models also reveal the basic hypothesis behind applied machine learning on text text is predictable. These low entropy phrases mean that language models would guess friend and broomstick, respectively, with a high likelihood and in fact, English speakers would be surprised if the phrase wasnt completed that way. Human listeners can use experience, imagination, and memory as well as sit uational context to fill in the blank. To take advantage of the predictability of text, we need to define a constrained, numeric decision space on which the model can compute. Text data provides many opportunities to extract features either at a shallow level by simply using string splitting, to deeper levels that parse text to extract morphological, syntactic, and even semantic representations from the data.


### Summarize page no. 270 (Index 269)

#### Asking the user to enter the page number

In [16]:
flag = True
while flag:
 pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 if pageNumber > (no_of_pages-1):
   print("\nInvalid page numer\n")
   pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 else:
   flag= False

Please enter the page number (from 0 to 331): 269


#### Printing the Page Content

In [17]:
# creating a page object
page = pdf_reader.getPage(pageNumber)
# extracting text from the page
pageText = page.extractText()
# printing the content of the page
print("Page Content:\n")
print(pageText)

Page Content:

simpler abstractions in the form of 
Pools and Queues, which we†ll explore in this
section.A common combination of both data and task parallelism is to
have two data parallel tasks; the first 
maps
 an operation to many
data inputs and the second 
reduces
 the map operation to a set of
aggregations. This style of parallel computing has been made very
popular by Hadoop and Spark, which we will discuss in the next
section.Larger workflows can be described as a directed acyclic graph (DAG), where a series
of parallel steps is executed with synchronization points in between. A synchroniza‡
tion point ensures that all parts of the processing have completed or caught up before
execution continues. Data is also generally exchanged at synchronization points, sent
out to parallel tasks from the main task, or retrieved by the main task.Data parallel execution is still appropriate for the Python multiprocessing library,
however, some additional considerations are required. The firs

#### Summerization, Printing Page Content After Cleaning and The splitted sentences

In [18]:
# summarization
summary = Summarize_Page(pdf_reader, pageNumber)

Page Content After Cleaning:

simpler abstractions in the form of Pools and Queues, which well explore in this section.A common combination of both data and task parallelism is to have two data parallel tasks the first maps an operation to many data inputs and the second reduces the map operation to a set of aggregations. This style of parallel computing has been made very popular by Hadoop and Spark, which we will discuss in the next section.Larger workflows can be described as a directed acyclic graph DAG, where a series of parallel steps is executed with synchronization points in between. A synchroniza tion point ensures that all parts of the processing have completed or caught up before execution continues. Data is also generally exchanged at synchronization points, sent out to parallel tasks from the main task, or retrieved by the main task.Data parallel execution is still appropriate for the Python multiprocessing library, however, some additional considerations are required. The

#### Printing Page Summary

In [19]:
# printing the summary of the page
print("Page Summary:\n")
print(summary)

Page Summary:

A common combination of both data and task parallelism is to have two data parallel tasks the first maps an operation to many data inputs and the second reduces the map operation to a set of aggregations. Data parallel execution is still appropriate for the Python multiprocessing library, however, some additional considerations are required. The first is how many pro cesses to use and how. Queue, a data structure that is both thread and multiprocessing safe because operations are synchronized with locks to ensure that only one process or thread has access to the queue at a time. The main process then can join the pool waiting for all processes to complete, or begin work immediately fetching data from the output queue for final processing.


### Summarize page no. 305 (Index 304)

#### Asking the user to enter the page number

In [20]:
flag = True
while flag:
 pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 if pageNumber > (no_of_pages-1):
   print("\nInvalid page numer\n")
   pageNumber = int(input("Please enter the page number (from 0 to %d): " %(no_of_pages-1)))
 else:
   flag= False

Please enter the page number (from 0 to 331): 304


#### Printing the Page Content

In [21]:
# creating a page object
page = pdf_reader.getPage(pageNumber)
# extracting text from the page
pageText = page.extractText()
# printing the content of the page
print("Page Content:\n")
print(pageText)

Page Content:

positively and negatively associated modifiers and nonlexical factors like sarcasm,
hyperbole, and symbolism.
In Chapter 1
 we briefly introduced sentiment analysis to describe the importance of
contextual features. Whereas a language feature like gender is often encoded in the
structure of language, sentiment is often much too complex to be encoded at the
token level. Take, for example, this sample text from a set of Amazon customer
reviews of patio, lawn, and garden equipment:
I used to use a really primitive manual edger that I inherited from my father. Blistered
hands and time wasted, I decided there had to be a better way and this is surely it. Edg‡
ing is one of those necessary evils if you want a great looking house. I don†t edge every
time I mow. Usually I do it every other time. The first time out after a long winter, edg‡
ing usually takes a little longer. After that, edging is a snap because you are basically in
maintenance mode. I also use this around my land

#### Summerization, Printing Page Content After Cleaning and The splitted sentences


In [22]:
# summarization
summary = Summarize_Page(pdf_reader, pageNumber)

Page Content After Cleaning:

positively and negatively associated modifiers and nonlexical factors like sarcasm, hyperbole, and symbolism. In Chapter 1 we briefly introduced sentiment analysis to describe the importance of contextual features. Whereas a language feature like gender is often encoded in the structure of language, sentiment is often much too complex to be encoded at the token level. Take, for example, this sample text from a set of Amazon customer reviews of patio, lawn, and garden equipment I used to use a really primitive manual edger that I inherited from my father. Blistered hands and time wasted, I decided there had to be a better way and this is surely it. Edg ing is one of those necessary evils if you want a great looking house. I dont edge every time I mow. Usually I do it every other time. The first time out after a long winter, edg ing usually takes a little longer. After that, edging is a snap because you are basically in maintenance mode. I also use this arou

#### Printing Page Summary

In [23]:
# printing the summary of the page
print("Page Summary:\n")
print(summary)

Page Summary:

Edg ing is one of those necessary evils if you want a great looking house. I dont edge every time I mow. we can see how positive and nega tive phrases modify each other, with the effect of sometimes inverting or magnifying the sentiment. The parse tree in Figure 12 4 illustrates how these syntactic chunks combine to influence the overall sentiment of the review.


## Closing the pdf file

In [0]:
pdf.close()

## Notes
##### We can change the following:
1. the number of clusters (here i choose to be the cleil of the square root of the number of sentences).
2. the minimum no. of words in the sentence to be included in the list to perform clustering.
3. the method of vectorization (here i choose TF universal sentence encoder as it gives good features but it can be any way of sentence embedding).
##### Also, there are some mistakes due to the library pypdf2 which can not exclud the footer or heading and also the outlines words are connected as in the first sentence in page 75"CHAPTER 4Text Vectorization and TransformationPipelinesMachine" and also the outline and the first sentence in the page; that's why I removed the last sentence as it always contains the footer of the page.
##### Also I avoided the pages that contain codes and images to be summerized.